Sentence count for puppies and kittens into a SQL database
=====================
2016 - U.S. Mainstream Media

In [23]:
import mediacloud
from API_config import MY_API_KEY

mc = mediacloud.api.MediaCloud(MY_API_KEY)
dtres = mc.sentenceCount('kittens', '+publish_date:[2016-01-01T00:00:00Z TO 2016-12-31T00:00:00Z} AND +tags_id_media:1')
kittens = dtres['count']
hcres = mc.sentenceCount('puppies', '+publish_date:[2016-01-01T00:00:00Z TO 2016-12-31T00:00:00Z} AND +tags_id_media:1')
puppies = hcres['count']

Importing SQLAlchemy and datetime
=====================

In [24]:
from sqlalchemy import *
import datetime

## Basic SQL Generation


### Creating a Table


In [25]:
# add `echo=True` to see log statements of all the SQL that is generated
engine = create_engine('sqlite:///:memory:',echo=True) # just save the db in memory for now (ie. not on disk)
metadata = MetaData()
# define a table to use
queries = Table('queries', metadata,
    Column('id', Integer, primary_key=True),
    Column('keywords', String(400), nullable=False),
    Column('sentences', Integer, nullable=False),
    Column('timestamp', DateTime, default=datetime.datetime.now),
)
metadata.create_all(engine) # and create the tables in the database

2016-12-08 11:06:24,692 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2016-12-08 11:06:24,695 INFO sqlalchemy.engine.base.Engine ()
2016-12-08 11:06:24,699 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2016-12-08 11:06:24,701 INFO sqlalchemy.engine.base.Engine ()
2016-12-08 11:06:24,704 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("queries")
2016-12-08 11:06:24,706 INFO sqlalchemy.engine.base.Engine ()
2016-12-08 11:06:24,710 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE queries (
	id INTEGER NOT NULL, 
	keywords VARCHAR(400) NOT NULL, 
	sentences INTEGER NOT NULL, 
	timestamp DATETIME, 
	PRIMARY KEY (id)
)


2016-12-08 11:06:24,711 INFO sqlalchemy.engine.base.Engine ()
2016-12-08 11:06:24,712 INFO sqlalchemy.engine.base.Engine COMMIT


### Inserting Data


In [26]:
insert_stmt = queries.insert()
str(insert_stmt) # see an example of what this will do

'INSERT INTO queries (id, keywords, sentences, timestamp) VALUES (:id, :keywords, :sentences, :timestamp)'

In [27]:
insert_stmt = queries.insert().values(keywords="puppies", sentences=puppies)
str(insert_stmt)

'INSERT INTO queries (keywords, sentences, timestamp) VALUES (:keywords, :sentences, :timestamp)'

In [28]:
db_conn = engine.connect()
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

2016-12-08 11:08:28,408 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, sentences, timestamp) VALUES (?, ?, ?)
2016-12-08 11:08:28,409 INFO sqlalchemy.engine.base.Engine ('puppies', 7456, '2016-12-08 11:08:28.408440')
2016-12-08 11:08:28,411 INFO sqlalchemy.engine.base.Engine COMMIT


[1]

In [29]:
insert_stmt = queries.insert().values(keywords="kittens", sentences=kittens)
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

2016-12-08 11:08:48,490 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, sentences, timestamp) VALUES (?, ?, ?)
2016-12-08 11:08:48,491 INFO sqlalchemy.engine.base.Engine ('kittens', 3980, '2016-12-08 11:08:48.490490')
2016-12-08 11:08:48,493 INFO sqlalchemy.engine.base.Engine COMMIT


[2]

### Retrieving Data


In [30]:
from sqlalchemy.sql import select
select_stmt = select([queries])
results = db_conn.execute(select_stmt)
for row in results:
    print row

2016-12-08 11:08:55,615 INFO sqlalchemy.engine.base.Engine SELECT queries.id, queries.keywords, queries.sentences, queries.timestamp 
FROM queries
2016-12-08 11:08:55,616 INFO sqlalchemy.engine.base.Engine ()
(1, u'puppies', 7456, datetime.datetime(2016, 12, 8, 11, 8, 28, 408440))
(2, u'kittens', 3980, datetime.datetime(2016, 12, 8, 11, 8, 48, 490490))


In [31]:
select_stmt = select([queries]).where(queries.c.id==1)
for row in db_conn.execute(select_stmt):
    print row

2016-12-08 11:09:08,101 INFO sqlalchemy.engine.base.Engine SELECT queries.id, queries.keywords, queries.sentences, queries.timestamp 
FROM queries 
WHERE queries.id = ?
2016-12-08 11:09:08,103 INFO sqlalchemy.engine.base.Engine (1,)
(1, u'puppies', 7456, datetime.datetime(2016, 12, 8, 11, 8, 28, 408440))


In [33]:
select_stmt = select([queries]).where(queries.c.keywords.like('k%'))
for row in db_conn.execute(select_stmt):
    print row

2016-12-08 11:09:21,768 INFO sqlalchemy.engine.base.Engine SELECT queries.id, queries.keywords, queries.sentences, queries.timestamp 
FROM queries 
WHERE queries.keywords LIKE ?
2016-12-08 11:09:21,769 INFO sqlalchemy.engine.base.Engine ('k%',)
(2, u'kittens', 3980, datetime.datetime(2016, 12, 8, 11, 8, 48, 490490))
